In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics -q
!pip install shapely -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.9 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from scipy.spatial import distance
model = YOLO('/content/best.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def sol(i1,i2):
    results1 = model(i1)
    results2 = model(i2)

    # Extract OBB data from predictions
    obb1 = results1[0].obb
    obb2 = results2[0].obb

    # Convert boxes (xywhr) and classes to NumPy arrays
    boxes1 = obb1.xywhr.cpu().numpy()  # [x_center, y_center, width, height, rotation]
    classes1 = obb1.cls.cpu().numpy()  # Class IDs (0 to 15)
    boxes2 = obb2.xywhr.cpu().numpy()
    classes2 = obb2.cls.cpu().numpy()

    # Initialize dictionaries for grouping boxes by class
    boxesup1_by_class = defaultdict(list)
    boxesup2_by_class = defaultdict(list)
    boxesdown1_by_class = defaultdict(list)
    boxesdown2_by_class = defaultdict(list)

    # Group boxes by class for image 1
    for i, c in enumerate(classes1):
      if int(c) % 2 == 0:
          boxesdown1_by_class[int(c)].append(boxes1[i])
      else:
          boxesup1_by_class[int(c)].append(boxes1[i])

    # Group boxes by class for image 2
    for i, c in enumerate(classes2):
      if int(c) % 2 == 0:
          boxesdown2_by_class[int(c)].append(boxes2[i])
      else:
          boxesup2_by_class[int(c)].append(boxes2[i])

    # Sort boxes within each class by x-coordinate
    for c in range(16):
      if c in boxesup1_by_class:
          boxesup1_by_class[c] = sorted(boxesup1_by_class[c], key=lambda box: box[0])
      if c in boxesup2_by_class:
          boxesup2_by_class[c] = sorted(boxesup2_by_class[c], key=lambda box: box[0])
      if c in boxesdown1_by_class:
          boxesdown1_by_class[c] = sorted(boxesdown1_by_class[c], key=lambda box: box[0])
      if c in boxesdown2_by_class:
          boxesdown2_by_class[c] = sorted(boxesdown2_by_class[c], key=lambda box: box[0])

    # Create pairs for "up" group (odd classes)
    pairs_up = []
    for c in range(1, 16, 2):
      if boxesup1_by_class[c] and boxesup2_by_class[c]:
          n = min(len(boxesup1_by_class[c]), len(boxesup2_by_class[c]))
          for i in range(n):
              box1 = boxesup1_by_class[c][i]
              box2 = boxesup2_by_class[c][i]
              idx1 = np.where((boxes1 == box1).all(axis=1))[0][0]
              idx2 = np.where((boxes2 == box2).all(axis=1))[0][0]
              pairs_up.append((box1, box2, idx1, idx2))

    # Create pairs for "down" group (even classes)
    pairs_down = []
    for c in range(0, 16, 2):
      if boxesdown1_by_class[c] and boxesdown2_by_class[c]:
          n = min(len(boxesdown1_by_class[c]), len(boxesdown2_by_class[c]))
          for i in range(n):
              box1 = boxesdown1_by_class[c][i]
              box2 = boxesdown2_by_class[c][i]
              idx1 = np.where((boxes1 == box1).all(axis=1))[0][0]
              idx2 = np.where((boxes2 == box2).all(axis=1))[0][0]
              pairs_down.append((box1, box2, idx1, idx2))

    # Function to calculate average shift between centroids
    def calculate_avg_shift(pairs):
      if not pairs:
          return np.array([0, 0], dtype=np.float32)
      centroids1 = np.array([box1[:2] for box1, _, _, _ in pairs], dtype=np.float32)
      centroids2 = np.array([box2[:2] for _, box2, _, _ in pairs], dtype=np.float32)
      return np.mean(centroids1 - centroids2, axis=0)

    # Calculate average shifts
    avg_shift_up = calculate_avg_shift(pairs_up)
    avg_shift_down = calculate_avg_shift(pairs_down)

    # Function to shift a box
    def shift_box(box, shift):
      shifted_box = box.copy()
      shifted_box[:2] += shift
      return shifted_box

    # Function to compute corner points from xywhr format
    def get_corners(xywhr):
      x, y, w, h, r = xywhr
      corners = [
          [x - w/2, y - h/2],
          [x + w/2, y - h/2],
          [x + w/2, y + h/2],
          [x - w/2, y + h/2]
      ]
      cos_r = np.cos(r)
      sin_r = np.sin(r)
      rotated_corners = []
      for px, py in corners:
          px -= x
          py -= y
          px_rot = px * cos_r - py * sin_r
          py_rot = px * sin_r + py * cos_r
          rotated_corners.append([px_rot + x, py_rot + y])
      return np.array(rotated_corners)

    # Collect all boxes for each group
    boxesup1 = [box for box, cls in zip(boxes1, classes1) if int(cls) % 2 != 0]  # Odd classes
    boxesup2 = [box for box, cls in zip(boxes2, classes2) if int(cls) % 2 != 0]
    boxesdown1 = [box for box, cls in zip(boxes1, classes1) if int(cls) % 2 == 0]  # Even classes
    boxesdown2 = [box for box, cls in zip(boxes2, classes2) if int(cls) % 2 == 0]

    # Compute shifted boxes for all boxes in image 2
    shifted_boxesup2 = [shift_box(box, avg_shift_up) for box in boxesup2]
    shifted_boxesdown2 = [shift_box(box, avg_shift_down) for box in boxesdown2]

    # Get corners for all boxes
    corners_box1_up = [get_corners(box) for box in boxesup1]
    corners_box2_up = [get_corners(box) for box in boxesup2]
    corners_shifted_box2_up = [get_corners(box) for box in shifted_boxesup2]
    corners_box1_down = [get_corners(box) for box in boxesdown1]
    corners_box2_down = [get_corners(box) for box in boxesdown2]
    corners_shifted_box2_down = [get_corners(box) for box in shifted_boxesdown2]

    # Function to calculate IoUs for paired boxes (unchanged)
    def calculate_ious(pairs, avg_shift, classes1, corners1_list, corners2_list, corners2_shifted_list):
      ious = []
      for idx, (box1, box2, idx1, idx2) in enumerate(pairs):
          corners1 = get_corners(box1)
          corners1_list.append(corners1)
          corners2 = get_corners(box2)
          corners2_list.append(corners2)
          shifted_box2 = shift_box(box2, avg_shift)
          corners2_shifted = get_corners(shifted_box2)
          corners2_shifted_list.append(corners2_shifted)
          poly1 = Polygon(corners1)
          poly2 = Polygon(corners2_shifted)
          intersection = poly1.intersection(poly2).area
          union = poly1.union(poly2).area
          iou = intersection / union if union > 0 else 0
          ious.append(iou)
          class_label = int(classes1[idx1])
          #print(f"IoU for pair {idx} (Class {class_label}): {iou:.4f}")
      return ious

    # Lists for IoU calculation (only for paired boxes)
    corners_for_box1_up = []
    corners_for_box2_up = []
    corners_for_box2_shifted_up = []
    corners_for_box1_down = []
    corners_for_box2_down = []
    corners_for_box2_shifted_down = []

    # Calculate IoUs
    #print("Up Group (Odd Classes):")
    ious_up = calculate_ious(pairs_up, avg_shift_up, classes1, corners_for_box1_up, corners_for_box2_up, corners_for_box2_shifted_up)
    #print("\nDown Group (Even Classes):")
    ious_down = calculate_ious(pairs_down, avg_shift_down, classes1, corners_for_box1_down, corners_for_box2_down, corners_for_box2_shifted_down)

    # Compute mean IoUs
    mean_iou_up = np.mean(ious_up) if ious_up else 0
    mean_iou_down = np.mean(ious_down) if ious_down else 0
    overall_ious = ious_up + ious_down
    mean_iou_overall = np.mean(overall_ious) if overall_ious else 0

    #print(f"\nMean IoU for up group (odd classes): {mean_iou_up:.4f} (based on {len(ious_up)} pairs)")
    #print(f"Mean IoU for down group (even classes): {mean_iou_down:.4f} (based on {len(ious_down)} pairs)")
    #print(f"Overall Mean IoU across all pairs: {mean_iou_overall:.4f} (based on {len(overall_ious)} pairs)")


    # Visualization with all boxes
    fig, axes = plt.subplots(2, 2, figsize=(18, 18))

    # Original Up Boxes
    axes[0, 0].set_title("Original Up Boxes (Odd Classes)")
    for box in corners_box1_up:
      box = np.vstack([box, box[0]])
      axes[0, 0].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_up[0] else "")
    for box in corners_box2_up:
      box = np.vstack([box, box[0]])
      axes[0, 0].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2' if box is corners_box2_up[0] else "")

    # Shifted Up Boxes
    axes[0, 1].set_title("Shifted Up Boxes (Odd Classes)")
    for box in corners_box1_up:
      box = np.vstack([box, box[0]])
      axes[0, 1].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_up[0] else "")
    for box in corners_shifted_box2_up:
      box = np.vstack([box, box[0]])
      axes[0, 1].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2 Shifted' if box is corners_shifted_box2_up[0] else "")

    # Original Down Boxes
    axes[1, 0].set_title("Original Down Boxes (Even Classes)")
    for box in corners_box1_down:
      box = np.vstack([box, box[0]])
      axes[1, 0].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_down[0] else "")
    for box in corners_box2_down:
      box = np.vstack([box, box[0]])
      axes[1, 0].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2' if box is corners_box2_down[0] else "")

    # Shifted Down Boxes
    axes[1, 1].set_title("Shifted Down Boxes (Even Classes)")
    for box in corners_box1_down:
      box = np.vstack([box, box[0]])
      axes[1, 1].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_down[0] else "")
    for box in corners_shifted_box2_down:
      box = np.vstack([box, box[0]])
      axes[1, 1].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2 Shifted' if box is corners_shifted_box2_down[0] else "")

    # Formatting
    for ax in axes.flatten():
      ax.set_xlabel("X")
      ax.set_ylabel("Y")
      ax.set_aspect('equal', adjustable='box')
      ax.invert_yaxis()
      ax.grid(True)
      if ax == axes[0, 0] or ax == axes[0, 1] or ax == axes[1, 0] or ax == axes[1, 1]:
          ax.legend()

    plt.tight_layout()
    plt.show()
    return mean_iou_overall

In [ ]:
x = 0
for i in range(1, 51):
    for j in range(i + 1, 51):  # Start j from i+1 to avoid (i, j) and (j, i)
        x = max(sol(f'/content/{i}.jpg', f'/content/{j}.jpg'), x)
print(x)

Streaming output truncated to the last 5000 lines.
Speed: 4.9ms preprocess, 617.7ms inference, 9.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/9.jpg: 640x640 614.9ms
Speed: 3.3ms preprocess, 614.9ms inference, 8.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/38.jpg: 640x640 611.9ms
Speed: 4.4ms preprocess, 611.9ms inference, 11.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/9.jpg: 640x640 633.7ms
Speed: 3.7ms preprocess, 633.7ms inference, 11.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/39.jpg: 640x640 624.1ms
Speed: 4.4ms preprocess, 624.1ms inference, 8.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/9.jpg: 640x640 605.7ms
Speed: 3.8ms preprocess, 605.7ms inference, 8.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/40.jpg: 640x640 608.4ms
Speed: 4.5ms preprocess, 608.4ms inference, 9.4ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
def sol1(i1, i2):
    results1 = model(i1)
    results2 = model(i2)

    # Extract OBB data from predictions
    obb1 = results1[0].obb
    obb2 = results2[0].obb

    # Convert boxes (xywhr) and classes to NumPy arrays
    boxes1 = obb1.xywhr.cpu().numpy()  # [x_center, y_center, width, height, rotation]
    classes1 = obb1.cls.cpu().numpy()  # Class IDs (0 to 15)
    boxes2 = obb2.xywhr.cpu().numpy()
    classes2 = obb2.cls.cpu().numpy()

    # Initialize dictionaries for grouping boxes by class
    boxesup1_by_class = defaultdict(list)
    boxesup2_by_class = defaultdict(list)
    boxesdown1_by_class = defaultdict(list)
    boxesdown2_by_class = defaultdict(list)

    # Group boxes by class for image 1
    for i, c in enumerate(classes1):
        if int(c) % 2 == 0:
            boxesdown1_by_class[int(c)].append(boxes1[i])
        else:
            boxesup1_by_class[int(c)].append(boxes1[i])

    # Group boxes by class for image 2
    for i, c in enumerate(classes2):
        if int(c) % 2 == 0:
            boxesdown2_by_class[int(c)].append(boxes2[i])
        else:
            boxesup2_by_class[int(c)].append(boxes2[i])

    # Function to compute corner points from xywhr format
    def get_corners(xywhr):
        x, y, w, h, r = xywhr
        corners = [
            [x - w / 2, y - h / 2],
            [x + w / 2, y - h / 2],
            [x + w / 2, y + h / 2],
            [x - w / 2, y + h / 2]
        ]
        cos_r = np.cos(r)
        sin_r = np.sin(r)
        rotated_corners = []
        for px, py in corners:
            px -= x
            py -= y
            px_rot = px * cos_r - py * sin_r
            py_rot = px * sin_r + py * cos_r
            rotated_corners.append([px_rot + x, py_rot + y])
        return np.array(rotated_corners)

    # Create pairs for "up" group (odd classes) using nearest-neighbor matching
    pairs_up = []
    for c in range(1, 16, 2):
        if boxesup1_by_class[c] and boxesup2_by_class[c]:
            centroids1 = np.array([box[:2] for box in boxesup1_by_class[c]])
            centroids2 = np.array([box[:2] for box in boxesup2_by_class[c]])
            dist_matrix = distance.cdist(centroids2, centroids1)
            for i in range(len(centroids2)):
                min_idx = np.argmin(dist_matrix[i])
                if dist_matrix[i, min_idx] < 50:  # Distance threshold
                    box2 = boxesup2_by_class[c][i]
                    box1 = boxesup1_by_class[c][min_idx]
                    idx1 = np.where((boxes1 == box1).all(axis=1))[0][0]
                    idx2 = np.where((boxes2 == box2).all(axis=1))[0][0]
                    pairs_up.append((box1, box2, idx1, idx2))

    # Create pairs for "down" group (even classes) using nearest-neighbor matching
    pairs_down = []
    for c in range(0, 16, 2):
        if boxesdown1_by_class[c] and boxesdown2_by_class[c]:
            centroids1 = np.array([box[:2] for box in boxesdown1_by_class[c]])
            centroids2 = np.array([box[:2] for box in boxesdown2_by_class[c]])
            dist_matrix = distance.cdist(centroids2, centroids1)
            for i in range(len(centroids2)):
                min_idx = np.argmin(dist_matrix[i])
                if dist_matrix[i, min_idx] < 50:  # Distance threshold
                    box2 = boxesdown2_by_class[c][i]
                    box1 = boxesdown1_by_class[c][min_idx]
                    idx1 = np.where((boxes1 == box1).all(axis=1))[0][0]
                    idx2 = np.where((boxes2 == box2).all(axis=1))[0][0]
                    pairs_down.append((box1, box2, idx1, idx2))

    # Function to calculate average shift between centroids
    def calculate_avg_shift(pairs):
        if not pairs:
            return np.array([0, 0], dtype=np.float32)
        centroids1 = np.array([box1[:2] for box1, _, _, _ in pairs], dtype=np.float32)
        centroids2 = np.array([box2[:2] for _, box2, _, _ in pairs], dtype=np.float32)
        return np.mean(centroids1 - centroids2, axis=0)

    # Calculate average shifts
    avg_shift_up = calculate_avg_shift(pairs_up)
    avg_shift_down = calculate_avg_shift(pairs_down)

    # Function to shift a box
    def shift_box(box, shift):
        shifted_box = box.copy()
        shifted_box[:2] += shift
        return shifted_box

    # Collect all boxes for each group
    boxesup1 = [box for box, cls in zip(boxes1, classes1) if int(cls) % 2 != 0]  # Odd classes
    boxesup2 = [box for box, cls in zip(boxes2, classes2) if int(cls) % 2 != 0]
    boxesdown1 = [box for box, cls in zip(boxes1, classes1) if int(cls) % 2 == 0]  # Even classes
    boxesdown2 = [box for box, cls in zip(boxes2, classes2) if int(cls) % 2 == 0]

    # Compute shifted boxes for all boxes in image 2
    shifted_boxesup2 = [shift_box(box, avg_shift_up) for box in boxesup2]
    shifted_boxesdown2 = [shift_box(box, avg_shift_down) for box in boxesdown2]

    # Get corners for all boxes
    corners_box1_up = [get_corners(box) for box in boxesup1]
    corners_box2_up = [get_corners(box) for box in boxesup2]
    corners_shifted_box2_up = [get_corners(box) for box in shifted_boxesup2]
    corners_box1_down = [get_corners(box) for box in boxesdown1]
    corners_box2_down = [get_corners(box) for box in boxesdown2]
    corners_shifted_box2_down = [get_corners(box) for box in shifted_boxesdown2]

    # Function to calculate IoUs for paired boxes
    def calculate_ious(pairs, avg_shift, classes1, corners1_list, corners2_list, corners2_shifted_list):
        ious = []
        for idx, (box1, box2, idx1, idx2) in enumerate(pairs):
            corners1 = get_corners(box1)
            corners1_list.append(corners1)
            corners2 = get_corners(box2)
            corners2_list.append(corners2)
            shifted_box2 = shift_box(box2, avg_shift)
            corners2_shifted = get_corners(shifted_box2)
            corners2_shifted_list.append(corners2_shifted)
            poly1 = Polygon(corners1)
            poly2 = Polygon(corners2_shifted)
            intersection = poly1.intersection(poly2).area
            union = poly1.union(poly2).area
            iou = intersection / union if union > 0 else 0
            ious.append(iou)
        return ious

    # Lists for IoU calculation (only for paired boxes)
    corners_for_box1_up = []
    corners_for_box2_up = []
    corners_for_box2_shifted_up = []
    corners_for_box1_down = []
    corners_for_box2_down = []
    corners_for_box2_shifted_down = []

    # Calculate IoUs
    ious_up = calculate_ious(pairs_up, avg_shift_up, classes1, corners_for_box1_up, corners_for_box2_up, corners_for_box2_shifted_up)
    ious_down = calculate_ious(pairs_down, avg_shift_down, classes1, corners_for_box1_down, corners_for_box2_down, corners_for_box2_shifted_down)

    # Compute mean IoUs
    mean_iou_up = np.mean(ious_up) if ious_up else 0
    mean_iou_down = np.mean(ious_down) if ious_down else 0
    overall_ious = ious_up + ious_down
    mean_iou_overall = np.mean(overall_ious) if overall_ious else 0
    """

    # Visualization with all boxes
    fig, axes = plt.subplots(2, 2, figsize=(18, 18))

    # Original Up Boxes
    axes[0, 0].set_title("Original Up Boxes (Odd Classes)")
    for box in corners_box1_up:
        box = np.vstack([box, box[0]])
        axes[0, 0].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_up[0] else "")
    for box in corners_box2_up:
        box = np.vstack([box, box[0]])
        axes[0, 0].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2' if box is corners_box2_up[0] else "")

    # Shifted Up Boxes
    axes[0, 1].set_title("Shifted Up Boxes (Odd Classes)")
    for box in corners_box1_up:
        box = np.vstack([box, box[0]])
        axes[0, 1].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_up[0] else "")
    for box in corners_shifted_box2_up:
        box = np.vstack([box, box[0]])
        axes[0, 1].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2 Shifted' if box is corners_shifted_box2_up[0] else "")

    # Original Down Boxes
    axes[1, 0].set_title("Original Down Boxes (Even Classes)")
    for box in corners_box1_down:
        box = np.vstack([box, box[0]])
        axes[1, 0].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_down[0] else "")
    for box in corners_box2_down:
        box = np.vstack([box, box[0]])
        axes[1, 0].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2' if box is corners_box2_down[0] else "")

    # Shifted Down Boxes
    axes[1, 1].set_title("Shifted Down Boxes (Even Classes)")
    for box in corners_box1_down:
        box = np.vstack([box, box[0]])
        axes[1, 1].plot(box[:, 0], box[:, 1], 'b-', lw=2, label='Image 1' if box is corners_box1_down[0] else "")
    for box in corners_shifted_box2_down:
        box = np.vstack([box, box[0]])
        axes[1, 1].plot(box[:, 0], box[:, 1], 'r-', lw=2, label='Image 2 Shifted' if box is corners_shifted_box2_down[0] else "")

    # Formatting
    for ax in axes.flatten():
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_aspect('equal', adjustable='box')
        ax.invert_yaxis()
        ax.grid(True)
        if ax == axes[0, 0] or ax == axes[0, 1] or ax == axes[1, 0] or ax == axes[1, 1]:
            ax.legend()

    plt.tight_layout()
    plt.show()
    """
    return mean_iou_overall

In [ ]:
import pandas as pd

# Create an empty list to store rows
rows = []


for i in range(50, 101, 5):  # 50 to 100, step 5
    threshold = i / 100
    match_count = 0
    matched_number = []
    for j in range(1,51):
        for k in range(1,51):
            x = sol1(f'/content/drive/MyDrive/img2/{j}.jpg', f'/content/drive/MyDrive/edited/{k}.jpg')
            if x >= threshold:
                match_count += 1
                matched_number.append([j,k])
    rows.append([threshold, match_count, matched_number])

# Convert to DataFrame
df = pd.DataFrame(rows, columns=['threshold', 'match count', 'matched number'])

# Save to CSV
df.to_csv('result.csv', index=False)

Streaming output truncated to the last 5000 lines.
image 1/1 /content/drive/MyDrive/edited/17.jpg: 640x640 508.6ms
Speed: 3.7ms preprocess, 508.6ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/img2/34.jpg: 640x640 553.5ms
Speed: 3.7ms preprocess, 553.5ms inference, 7.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/edited/18.jpg: 640x640 520.6ms
Speed: 3.6ms preprocess, 520.6ms inference, 5.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/img2/34.jpg: 640x640 531.6ms
Speed: 3.7ms preprocess, 531.6ms inference, 7.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/edited/19.jpg: 640x640 512.7ms
Speed: 5.3ms preprocess, 512.7ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/img2/34.jpg: 640x640 513.6ms
Speed: 3.7ms preprocess, 513.6ms inference, 7.2ms postprocess per image at s

In [ ]:
import pandas as pd

# Create an empty list to store rows
rows = []
nums = [2,7,12,15,22,25,32,38,43,50]

for i in range(50, 101, 5):  # 50 to 100, step 5
    threshold = i / 100
    match_count = 0
    matched_number = []
    for j in nums:
        for k in nums:
            for l in range(3):
                x = sol1(f'/content/drive/MyDrive/img1/{j}.jpg', f'/content/drive/MyDrive/edt/{k}e{l+1}.jpg')
                if x >= threshold:
                    match_count += 1
                    matched_number.append([j,k])
    rows.append([threshold, match_count, matched_number])

# Convert to DataFrame
df = pd.DataFrame(rows, columns=['threshold', 'match count', 'matched number'])

# Save to CSV
df.to_csv('result.csv', index=False)

Streaming output truncated to the last 5000 lines.
image 1/1 /content/drive/MyDrive/edt/12e1.jpg: 640x640 952.4ms
Speed: 8.8ms preprocess, 952.4ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/img1/12.jpg: 640x640 1023.2ms
Speed: 8.2ms preprocess, 1023.2ms inference, 16.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/edt/12e2.jpg: 640x640 719.8ms
Speed: 7.0ms preprocess, 719.8ms inference, 8.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/img1/12.jpg: 640x640 644.1ms
Speed: 6.6ms preprocess, 644.1ms inference, 11.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/edt/12e3.jpg: 640x640 674.7ms
Speed: 8.3ms preprocess, 674.7ms inference, 11.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/img1/12.jpg: 640x640 648.5ms
Speed: 8.7ms preprocess, 648.5ms inference, 10.8ms postprocess per image 